# **Case Study #2 - Pizza Runner - Solutions**

**Observations:**

In customer\_order tables, below row repeated twice, I am assuming that customer might have ordered the pizza 2 times as same kind of  pizza with same exclusions at the same time.

In [4]:
SELECT
*
FROM customer_orders
WHERE order_id =4  and pizza_id=1

(2 row(s) affected)

Total execution time: 00:00:00.003

order_id,customer_id,pizza_id,exclusions,extras,order_time
4,103,1,4,NULL,2020-01-04T13:23:46
4,103,1,4,NULL,2020-01-04T13:23:46


Cleaning the customer\_orders table and creating a view for further analysis

In [6]:
DROP VIEW IF EXISTS new_customer_orders;
CREATE VIEW new_customer_orders AS
		SELECT  
			order_id,
			customer_id,
            pizza_id,
            CASE
                WHEN exclusions = '' THEN NULL
                WHEN exclusions = 'null' THEN NULL
                ELSE exclusions
            END AS exclusions,
            CASE
                WHEN extras = '' THEN NULL
                WHEN extras = 'null' THEN NULL
                WHEN extras = 'NaN' THEN NULL
                ELSE extras
            END AS extras,
            order_time
    FROM customer_orders;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.033

Cleaning the runners\_orders table and creating a view for further analysis

In [9]:
DROP VIEW IF EXISTS new_runner_orders;
CREATE VIEW new_runner_orders AS
SELECT
	order_id,
    runner_id,
    CASE
		WHEN pickup_time = 'null' THEN NULL
		ELSE pickup_time
	END AS pickup_time,
	CAST(
		CASE
			WHEN distance = 'null' THEN NULL
			ELSE REGEXP_REPLACE(distance,'[a-z]+', '')
		END AS DECIMAL
        ) AS distance,
	CAST(
		CASE 
			WHEN duration = 'null' THEN NULL
			ELSE REGEXP_REPLACE(duration, '[a-z]+', '')
		END AS DECIMAL 
        )  AS duration,
	CASE
		WHEN cancellation ='null' OR cancellation = 'NaN' OR cancellation = '' THEN NULL
		ELSE cancellation
	END AS cancellation
FROM runner_orders;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.006

# **A. Pizza Metrics**

## 1\. How many pizzas were ordered?

In [11]:
SELECT 
	count(order_id) as Total_pizzas_ordered
FROM new_customer_orders;

(1 row(s) affected)

Total execution time: 00:00:00.003

total_pizzas_ordered
14


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">2. How many unique customer orders were made?</span>

In [12]:
SELECT 
	count(DISTINCT(customer_id)) as unique_customers
FROM new_customer_orders;

(1 row(s) affected)

Total execution time: 00:00:00.002

unique_customers
5


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">3. How many successful orders were delivered by each runner?</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">Not considering the cancellation orders</span>

In [13]:
SELECT
	runner_id,
    count(*) as successful_orders_delivered
FROM new_runner_orders
WHERE cancellation is null
GROUP BY runner_id;

(3 row(s) affected)

Total execution time: 00:00:00.002

runner_id,successful_orders_delivered
1,4
2,3
3,1


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">4. How many of each type of pizza was delivered?</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">counting only delivered pizzas</span>

In [14]:
SELECT 
	p.pizza_name,
    count(*) as count
FROM new_customer_orders c
JOIN pizza_names p
	ON c.pizza_id = p.pizza_id
JOIN new_runner_orders r
	ON c.order_id = r.order_id
    WHERE cancellation is null
	GROUP BY p.pizza_name;

(2 row(s) affected)

Total execution time: 00:00:00.004

pizza_name,count
Meatlovers,9
Vegetarian,3


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">5. How many Vegetarian and Meatlovers were ordered by each customer?</span>

<span style="color: rgb(64, 64, 64); background-color: rgb(255, 255, 255);">use case statement to count the Vegetarian and Meatlovers for each customer</span>

In [15]:
SELECT
	c.customer_id,
    COUNT(CASE
		WHEN p.pizza_name = 'Meatlovers' THEN 1
	END) as meatlovers_orders,
    COUNT(CASE
		WHEN p.pizza_name = 'Vegetarian' THEN 1
	END) as Vegetarian_orders
FROM new_customer_orders c
JOIN pizza_names p
	ON c.pizza_id = p.pizza_id
    GROUP BY c.customer_id;

(5 row(s) affected)

Total execution time: 00:00:00.003

customer_id,meatlovers_orders,vegetarian_orders
101,2,1
103,3,1
104,3,0
105,0,1
102,2,1


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">6. What was the maximum number of pizzas delivered in a single order?</span>

In [16]:
SELECT
	c.order_id,
    count(*) as pizzas_delivered_count
FROM new_customer_orders c
JOIN new_runner_orders r
	ON c.order_id = r.order_id
    WHERE cancellation is null
    GROUP BY c.order_id
    ORDER BY pizzas_delivered_count desc
    LIMIT 1;

(1 row(s) affected)

Total execution time: 00:00:00.001

order_id,pizzas_delivered_count
4,3


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?</span>

<span style="background-color: rgb(255, 255, 255); font-size: 14px;"><font color="#404040"><div style=""><ul><li>at least 1 change means exclusions in pizza</li><li>no change means no exclusions in pizza</li></ul></div></font></span>

In [17]:
SELECT
	c.customer_id,
    COUNT(*) as total_delivered_pizzas,
    COUNT(case
		WHEN c.exclusions is not null or c.extras is not null THEN 1
	END)as atleat_one_change,
    COUNT(case
		WHEN c.exclusions is null and c.extras is null THEN 1
	END)as no_changes
FROM new_customer_orders c
JOIN new_runner_orders r
	ON c.order_id = r.order_id
    WHERE cancellation is null
    GROUP BY c.customer_id;

(5 row(s) affected)

Total execution time: 00:00:00.003

customer_id,total_delivered_pizzas,atleat_one_change,no_changes
101,2,0,2
102,3,0,3
103,3,3,0
104,3,2,1
105,1,1,0


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">8. How many pizzas were delivered that had both exclusions and extras?</span>

In [18]:
SELECT
    count(*) AS pizzas_with_exclusions_and_extras
FROM new_customer_orders c
JOIN new_runner_orders r
	ON c.order_id = r.order_id
    WHERE r.cancellation is null
    AND c.exclusions is not null
    AND c.extras is not null

(1 row(s) affected)

Total execution time: 00:00:00.003

pizzas_with_exclusions_and_extras
1


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">9. What was the total volume of pizzas ordered for each hour of the day?</span>

<span style="background-color: rgb(255, 255, 255);"><font color="#404040"><span style="font-size: 14px;">extract hour from order_time using EXTRACT()</span></font><br></span>

In [19]:
SELECT
	EXTRACT(HOUR FROM order_time) as Hour,
	COUNT(*)
FROM new_customer_orders
GROUP BY Hour
ORDER BY Hour;

(7 row(s) affected)

Total execution time: 00:00:00.003

hour,count
11.0,1
12.0,2
13.0,3
18.0,3
19.0,1
21.0,3
23.0,1


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">10. What was the volume of orders for each day of the week?</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">used the to_char() to get the weekday of the date</span>

<span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit;">group by weekday</span>

In [20]:
SELECT
	to_char(order_time, 'Day') AS weekday,
	COUNT(*)
FROM new_customer_orders
GROUP BY weekday;

(4 row(s) affected)

Total execution time: 00:00:00.002

weekday,count
Saturday,5
Thursday,3
Friday,1
Wednesday,5
